<img src='img/logo.png'>
<img src='img/title.png'>

# Pipelines

This notebook covers "pipelines" or machine learning workflows that are expressed as a sequence of named steps with parameters that may be varied in a grid or randomized search.

# Table of Contents
* [Pipelines](#Pipelines)
	* [Algorithm chains and pipelines](#Algorithm-chains-and-pipelines)
		* [Building pipelines](#Building-pipelines)
			* [Another Pipeline example](#Another-Pipeline-example)
			* [And here it is with a Pipeline](#And-here-it-is-with-a-Pipeline)
		* [The General Pipeline Interface](#The-General-Pipeline-Interface)
		* [Pipeline attributes](#Pipeline-attributes)
			* [Access by name](#Access-by-name)
* [Summary](#Summary)


In [ ]:
#Fix working directory
%cd notebooks

In [ ]:
import numpy as np
import pandas as pd

import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['image.interpolation'] = "none"
np.set_printoptions(precision=3)

import src.mglearn as mglearn

## Algorithm chains and pipelines

The next few cells show a pipeline workflow written out in declarative form with the tools we have covered so far, including `MinMaxScaler` for scaling, and `GridSearchCV` for model selection with cross validation.

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# load and split the data
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0)

In [ ]:
# compute minimum and maximum on the training data
scaler = MinMaxScaler().fit(X_train)
# rescale training data
X_train_scaled = scaler.transform(X_train)

In [ ]:
svm = SVC(gamma='auto')
# learn an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
# scale test data and score the scaled data
X_test_scaled = scaler.transform(X_test)
svm.score(X_test_scaled, y_test)

### Building pipelines

`pipe = make_pipeline(` in the next cell is a shorter way of expressing the logic in the cells above.  It creates a `Pipeline` with two named steps, `'minmaxascaler'` and `'svc'`.

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(MinMaxScaler(), SVC(gamma='auto'))
pipe

In [ ]:
# pipe has an interface like SVC() but the "fit" method is inclusive of pipeline steps
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

#### Another Pipeline example

Let's start with 100 rows and 10,000 columns of random data.

In [ ]:
rnd = np.random.RandomState(seed=0)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

`SelectPercentile` feature selctor as first step, the `Ridge` regression (regression with `L2` norm penalty).

Select the top 5% of the features with the lowest f_regression sore.


In [ ]:
from sklearn.feature_selection import SelectPercentile, f_regression

select = SelectPercentile(score_func=f_regression, percentile=5).fit(X, y)
X_selected = select.transform(X)
print(X_selected.shape)

To perform cross validation we need to fit and transform each fold separately.

First, here's how it would be done delcaratively:

In [ ]:
from sklearn.model_selection import KFold

kfold5 = KFold(5, random_state=0)

In [ ]:
from sklearn.linear_model import Ridge

scores = np.empty(0)
for train, test in kfold5.split(X):
    X_train = X[train]
    y_train = y[train]
    X_test  = X[test]
    y_test  = y[test]
    
    selector = SelectPercentile(score_func=f_regression, percentile=5).fit(X_train, y_train)
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    score = Ridge().fit(X_train_selected, y_train).score(X_test_selected, y_test)
    scores = np.append(scores, score)
    
    
np.mean(scores)

#### And here it is with a Pipeline

In [ ]:
from sklearn.model_selection import cross_val_score

pipe = make_pipeline(SelectPercentile(score_func=f_regression, percentile=5), Ridge())
scores = cross_val_score(pipe, X, y, cv=kfold5.split(X))
scores.mean()

### The General Pipeline Interface

<img src="img/pipeline-diagram.png" align='right' alt="Pipeline Illustration" width="43%"/>

```python
def fit(self, X, y):
    X_transformed = X
    for step in self.steps[:-1]:
        # iterate over all but the final step
        # fit and transform the data
        X_transformed = step[1].fit_transform(X_transformed, y)
    # fit the last step
    self.steps[-1][1].fit(X_transformed, y)
    return self

def predict(self, X):
    X_transformed = X
    for step in self.steps[:-1]:
        # iterate over all but the final step
        # transform the data
        X_transformed = step[1].transform(X_transformed)
    # fit the last step
    return self.steps[-1][1].predict(X_transformed)
```

Image: CC-BY-NA, [Karl Rosaen](http://karlrosaen.com/ml/learning-log/2016-06-20/)

### Pipeline attributes

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), SVC(gamma='auto', C=100))
pipe.steps

#### Access by name

In [ ]:
pipe.fit(cancer.data, cancer.target)

components = pipe.named_steps["pca"].components_
print(components.shape)

# Summary

In this notebook, we reviewed the following topics in preparation for more advanced topics:

 * [Algorithm chains and pipelines](#Algorithm-Chains-and-Pipelines)
 * [The general pipeline interface](#The-General-Pipeline-Interface)
 * [Accessing step attributes](#Accessing-step-attributes)

<img src='img/copyright.png'>